## Import the required packages

In [15]:
import os
import sys
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import math
import json
import pandas as pd

## Load the data

In [16]:
fname = "../data-in/ibadan_EAs_of_interest_v2/shapefile_of_Ibadan_EAs_of_interest_v2.shx"
ib_data = gpd.read_file(fname)

In [17]:
ib_data.head()

,city,ward,ea_code,EAS,Longitude,Latitude,Settlement,geometry
0,Ibadan,Agugu,Ag_001,Opposite Christ the king primary school I,3.919976,7.386031,Slum,POINT (3.91998 7.38603)
1,Ibadan,Agugu,Ag_002,Beside Christ the King Primary School I,3.920514,7.386621,Slum,POINT (3.92051 7.38662)
2,Ibadan,Agugu,Ag_003,Behind Christ the kind Primary School I,3.920634,7.385972,Slum,POINT (3.92063 7.38597)
3,Ibadan,Agugu,Ag_004,Behind The Board of Trade,3.922894,7.386947,Slum,POINT (3.92289 7.38695)
4,Ibadan,Agugu,Ag_005,Opposite The Board of Trade,3.922421,7.386521,Slum,POINT (3.92242 7.38652)


In [18]:
ib_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   city        378 non-null    object  
 1   ward        378 non-null    object  
 2   ea_code     378 non-null    object  
 3   EAS         378 non-null    object  
 4   Longitude   378 non-null    float64 
 5   Latitude    378 non-null    float64 
 6   Settlement  378 non-null    object  
 7   geometry    378 non-null    geometry
dtypes: float64(2), geometry(1), object(5)
memory usage: 23.8+ KB


## Helper functions

In [4]:
def generate_rectangle(center_lon, center_lat, width, height):
    """
    Generate the coordinates of a rectangle centered around a given location.
    
    Parameters:
    - center_lon: Longitude of the center point
    - center_lat: Latitude of the center point
    - width: Width of the rectangle in meters
    - height: Height of the rectangle in meters
    
    Returns:
    - List of coordinates representing the rectangle
    """
    # Convert meters to degrees (approximate)
    meters_to_degrees_lon = (111320 * math.cos(center_lat * math.pi / 180))
    meters_to_degrees_lat = 110574
    
    # Calculate the coordinates of the rectangle corners
    min_lon = center_lon - (width / (2 * meters_to_degrees_lon))
    max_lon = center_lon + (width / (2 * meters_to_degrees_lon))
    min_lat = center_lat - (height / (2 * meters_to_degrees_lat))
    max_lat = center_lat + (height / (2 * meters_to_degrees_lat))
    
    # Define the rectangle coordinates in counterclockwise direction
    rectangle_coordinates = [
        [min_lon, min_lat],
        [min_lon, max_lat],
        [max_lon, max_lat],
        [max_lon, min_lat],
        [min_lon, min_lat]  # Closing the loop
    ]
    
    return rectangle_coordinates

# Example usage:
# Given location

# center_lon = 3.951372252965484
# center_lat = 7.434130780421128

# # Generate rectangle with width and height of 20 meters
# rectangle_coordinates = generate_rectangle(center_lon, center_lat, 20, 20)
# print(rectangle_coordinates)

In [20]:
def df_to_json(df, settings):
    """
    Convert a DataFrame with longitude and latitude columns to a JSON format with specified structure.
    
    Parameters:
    - df: DataFrame containing the data
    - setting: include all the feature that need to be included. 
    - polygon: polygon the polygon shape that needs to be created based on the given latitude and longitude
    
    Returns:
    - JSON formatted data
    """
    features = []
    coordinates = settings['extracted_features']
    properties = settings['properties_to_include']
    sparcity = settings['distance']
    
    for _, row in df.iterrows():
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        if properties:
            props = {prop: row[prop] for prop in properties}
        else:
            props = {}
        
        # Add longitude and latitude to properties
        props[coordinates[0]] = lon
        props[coordinates[1]] = lat
        if settings['shape_type'] == 'Polygon' or settings['shape_type'] == 'Both': 
            shape = [generate_rectangle(lon, lat, sparcity['width'], sparcity['height'])]
            # Create GeoJSON feature
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Polygon",
                    "coordinates": shape
                }
            }
            features.append(feature)
        if settings['shape_type'] == "Point" or settings['shape_type'] == 'Both': 
            shape = [lon, lat]
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Point",
                    "coordinates": shape
                }
            }


            features.append(feature)
    
    # Create FeatureCollection
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }
    
    return feature_collection

In [21]:
import geojson

def df_to_geojson(df, settings):
    """
    Convert a DataFrame with longitude and latitude columns to a GeoJSON format with specified structure.
    
    Parameters:
    - df: DataFrame containing the data
    - setting: include all the feature that need to be included. 
    - polygon: polygon the polygon shape that needs to be created based on the given latitude and longitude
    
    Returns:
    - GeoJSON FeatureCollection
    """
    features = []
    coordinates = settings['extracted_features']
    properties = settings['properties_to_include']
    sparcity = settings['distance']
    
    for _, row in df.iterrows():
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        if properties:
            props = {prop: row[prop] for prop in properties}
        else:
            props = {}
        
        # Add longitude and latitude to properties
        props[coordinates[0]] = lon
        props[coordinates[1]] = lat
        if settings['shape_type'] == 'Polygon' or settings['shape_type'] == 'Both': 
            shape = [generate_rectangle(lon, lat, sparcity['width'], sparcity['height'])]
            # Create GeoJSON feature
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Polygon",
                    "coordinates": shape
                }
            }
            features.append(feature)
        if settings['shape_type'] == "Point" or settings['shape_type'] == 'Both': 
            shape = [lon, lat]
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Point",
                    "coordinates": shape
                }
            }


            features.append(feature)
    
    # Create FeatureCollection
    # feature_collection = {
    #     "type": "FeatureCollection",
    #     "features": features
    # }
    # Create FeatureCollection
    feature_collection = geojson.FeatureCollection(features)
    
    return feature_collection

### Apply to Ibadan data

**Example usage:**

* Assuming your DataFrame is named 'df' and contains columns: city, ward, ea_code, EAS, Longitude, Latitude, Settlement, geometry

* And you want to include city, ward, ea_code, EAS, Settlement in the properties


#### JSON Files

In [35]:
### Create JSON data
settings = {"properties_to_include": ["city", "ward", 'ea_code', 'EAS', 'Settlement'],
            "extracted_features": ['Longitude', 'Latitude'],
            "distance": {'width': 100, 'height': 100}, # Width and height are messured in meters. 
            "shape_type": 'Polygon' # Polygone, Point, Both
}
 
json_data = df_to_json(ib_data, settings)

### Save JSON to a file
with open('../data-in/json-files/ibadan_json_file_c100m.json', 'w') as f:
    json.dump(json_data, f)

In [28]:
# json_data

#### GeoJSON Files

In [27]:
settings = {
    "properties_to_include" : ["city", "ward", 'ea_code', 'EAS', 'Settlement'],
    'extracted_features' : ['Longitude', 'Latitude'],
    "distance" : {'width' : 10, 'height': 10}, # Width and height are messured in miters. 
    'shape_type' : 'Polygon' # Polygone, Point, Both
}

geojson_data = df_to_geojson(ib_data, settings)
# Save GeoJSON to a file
with open('../data-in/json-files/ibadan_json_file_c10m.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

    

In [29]:
# geojson_data